# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [1]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("./data/soil_measures.csv")
print(crops.shape)
crops.head()

(2200, 5)


,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


## Class balance

From the counts below, it looks like we have perfect balance between the classes.

In [2]:
crops['crop'].value_counts()

crop
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

## Missing values

Doesn't look like there are any missing values.

In [3]:
print(crops.isna().sum().sort_values())

N       0
P       0
K       0
ph      0
crop    0
dtype: int64


## Encode the target column

The `crop` column is categorical and needs to be encoded.

In [4]:
crops['crop'] = crops['crop'].astype("category")
crops['crop_code'] = crops['crop'].cat.codes
print(crops.head(), '\n')
print(crops.tail(), '\n')
crops['crop_code'].value_counts()

    N   P   K        ph  crop  crop_code
0  90  42  43  6.502985  rice         20
1  85  58  41  7.038096  rice         20
2  60  55  44  7.840207  rice         20
3  74  35  40  6.980401  rice         20
4  78  42  42  7.628473  rice         20 

        N   P   K        ph    crop  crop_code
2195  107  34  32  6.780064  coffee          5
2196   99  15  27  6.086922  coffee          5
2197  118  33  30  6.362608  coffee          5
2198  117  32  34  6.758793  coffee          5
2199  104  18  30  6.779833  coffee          5 



crop_code
20    100
11    100
8     100
6     100
4     100
17    100
16    100
0     100
15    100
21    100
7     100
12    100
1     100
19    100
10    100
2     100
14    100
13    100
18    100
9     100
3     100
5     100
Name: count, dtype: int64

## Strategy

### Metric

The *negative log likelihood* or *cross-entropy* is used as the loss function for logistic regression in scikit, so this would be the most natural choice for the metric we use to evaluate model performance.

### Workflow

1. Build a set of single parameter logistic regression models
2. Evaluate each set of models using 5-Fold CV and cross-entropy as the metric
3. Create a boxplot with each predictor on the x-axis and cross-entropy of the y-axis

The boxplot will allow us to see how each of the single-predictor models relative to each other. From this plot, the best model will be selected as the parameter with the most importance feature for predictive performance.

In [5]:
rand_state = 711  # for reproducibility
X = crops.drop(["crop", "crop_code"], axis="columns")
y = crops['crop_code']

# split to train and test sets - dol BEFORE scaling
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rand_state)

X.head()

,N,P,K,ph
0,90,42,43,6.502985
1,85,58,41,7.038096
2,60,55,44,7.840207
3,74,35,40,6.980401
4,78,42,42,7.628473


In [6]:
X_train.head()

,N,P,K,ph
1830,16,6,29,5.868285
1573,7,141,195,5.514253
386,17,64,17,5.662699
141,60,44,23,5.722580
2044,94,37,41,6.463539


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# check to unscaled values
print(X.mean(), '\n')
print(X.std(), '\n')
# check to scaled values - note: scaling converts to numpy array
print(f"X_train_scaled type is {type(X_train_scaled)}, shape is {X_train_scaled.shape}")
print(f"X_test_scaled type is {type(X_test_scaled)}, shape is {X_test_scaled.shape}  \n")

print(X_train_scaled.mean(axis=0))  # 4 value that should be close to 0
print(X_train_scaled.std(axis=0))   # 4 value that should be close to 1

N     50.551818
P     53.362727
K     48.149091
ph     6.469480
dtype: float64 

N     36.917334
P     32.985883
K     50.647931
ph     0.773938
dtype: float64 

X_train_scaled type is <class 'numpy.ndarray'>, shape is (1650, 4)
X_test_scaled type is <class 'numpy.ndarray'>, shape is (550, 4)  

[1.29189588e-17 8.28966525e-17 0.00000000e+00 5.16758353e-16]
[1. 1. 1. 1.]


In [8]:
cv_folds = 5      # specified above

# test building and testing a single model
predictors = list(crops.columns)[:-1]

pred = predictors[0]  # N
kf = KFold(n_splits=cv_folds, random_state=rand_state, shuffle=True)
logreg_model = LogisticRegression()
# get the predictor vector and scale it
X_train_scaled = scaler.fit_transform(X_train[pred].to_numpy().reshape(-1, 1))
X_test_scaled = scaler.fit_transform(X_test[pred].to_numpy().reshape(-1, 1))
# get encoded target vector
y_train_vec = y_train.values
y_test_vec = y_test.values
print(f"shape of scaled training predictor vector: {X_train_scaled.shape} | shape of target training target vector: {y_train_vec.shape}")

# fit on a single predictor
logreg_model.fit(X_train_scaled, y_train_vec)
# predict on the test set
preds = logreg_model.predict(X_test_scaled)
print(logreg_model.score(X_test_scaled, y_test_vec))

shape of scaled training predictor vector: (1650, 1) | shape of target training target vector: (1650,)
0.10727272727272727


In [ ]:
import matplotlib.pyplot as plt


# print(predictors)  # ['N', 'P', 'K', 'ph']
results = []
mod_labels = []


In [ ]:
# test a single model build


pred = predictors[0]  # N
kf = KFold(n_splits=cv_folds, random_state=rand_state, shuffle=True)
logreg_model = LogisticRegression()
X_train_vec = X_train[pred].to_numpy().reshape(-1, 1)
y_train_vec = y_train.values
print(f"shape of training predictor vector: {X_train_vec.shape} | shape of training target vector: {y_train_vec.shape}")

# fit on a single predictor
logreg_model.fit(X_train_vec, y_train_vec)
# y_pred = model.predict(X_test[pred])

# print(type(y_pred))

In [ ]:
for pred in predictors:
    kf = KFold(n_splits=cv_folds, random_state=rand_state, shuffle=True)
    model = LogisticRegression()
    X_train_pred = X_train[pred]
    cv_results = cross_val_score(estimator=model, X=X_train_pred, y=y_train.values, scoring='neg_log_loss', cv=kf)
    mod_labels.append(pred + " predictor")
    results.append(cv_results)

plt.boxplot(results, labels=mod_labels)